In [ ]:
import os
import time
import numpy as np
from PIL import Image
from scipy import ndimage, stats
from scipy.fft import fft, fftfreq
import skimage.io
from skimage.measure import regionprops, label, find_contours
from skimage.segmentation import flood
import matplotlib.pyplot as plt
from scipy.ndimage import shift, distance_transform_edt, rotate, map_coordinates, zoom
import tifffile as tiff
import cv2
import glob
from scipy.signal import wiener, find_peaks
from circle_fit import taubinSVD, plot_data_circle, hyperSVD, prattSVD, hyperLSQ, lm, standardLSQ
import importlib
import fourierslice_refocusing
importlib.reload(fourierslice_refocusing)
import new_hilo1
importlib.reload(new_hilo1)

In [ ]:
def pre_processing(angle, pitch, file_path, bg_path=None):
   start = time.perf_counter()
   LFM1 = skimage.io.imread(file_path)
   if bg_path != None:
       bg = skimage.io.imread(bg_path)
       LFM1 = (LFM1 - bg)
   img_rotated = ndimage.rotate(LFM1, angle, reshape=True) 
   LFM1 = img_rotated.astype(np.int32)
   h, w = LFM1.shape[:2]
   fraction1 = 15/pitch
   fraction2 = 15/pitch
   img_resized = np.array(Image.fromarray(LFM1, mode='I').resize((int(fraction1 * w), int(fraction2 * h)), Image.Resampling.BILINEAR))
   end = time.perf_counter()
   print(f'Pre-processing took: {end - start}')
   return img_resized

# Number of microlenses in X and Y on the raw image (140 x 140 for us)
Nx=140
Ny=140

def calibrate_LFM3(img, x, y):
    start = time.perf_counter()
    step = 15  # espacement entre microlentilles

    LFM = np.zeros((Nx, Ny, step, step))
    XX = np.zeros((Nx, Ny))
    YY = np.zeros((Nx, Ny))
    CenterX = np.zeros((Nx, Ny))
    CenterY = np.zeros((Nx, Ny))
    CenterX_ini = np.zeros((Nx, Ny))
    CenterY_ini = np.zeros((Nx, Ny))

    for ii in range(Nx):
        for jj in range(Ny):

            if ii == 0 and jj == 0:
                cx, cy = x, y
            elif ii == 0:
                # première ligne : moyenne impossible, on prend celle de gauche
                cx = CenterX[ii, jj - 1] + step
                cy = CenterY[ii, jj - 1]
            elif jj == 0:
                # premiere colonne : moyenne impossible, se base sur la lentille au-dessus
                cx = CenterX[ii - 1, jj]
                cy = CenterY[ii - 1, jj] + step
            else:
                # moyenne entre la lentille de gauche et celle du dessus
                cx = (CenterX[ii - 1, jj] + CenterX[ii, jj - 1] + step) / 2
                cy = (CenterY[ii - 1, jj] + CenterY[ii, jj - 1] + step) / 2

            cx = int(cx)
            cy = int(cy)

            # extraction de la microlentille dans l'image
            intermediate = img[cy-(step//2):cy+(step//2 + 1), cx-(step//2):cx+(step//2 + 1)]
            intermediate = intermediate.astype(float)
            
            max_pos = np.unravel_index(np.argmax(intermediate), intermediate.shape)
            thresh = np.percentile(intermediate, 30)
            flooded = flood(intermediate, seed_point=max_pos, tolerance=intermediate[max_pos]-thresh)

            ##### Contours + Circle-fit #####
            flooded = flooded.astype(np.uint8)
            contours, _ = cv2.findContours(flooded, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
            main_contour = min(contours, key=len)
            points = main_contour[:, 0, :]  # de (n,1,2) vers (n,2)
            fit = taubinSVD(points)
            ymax = round(fit[1])
            xmax = round(fit[0])
            
            dx = xmax - (step // 2)
            dy = ymax - (step // 2)

            XX[ii, jj] = dx
            YY[ii, jj] = dy

            # ajustement du centre
            cx_adj = cx + dx
            cy_adj = cy + dy
            CenterX_ini[ii, jj] = cx
            CenterY_ini[ii, jj] = cy
            CenterX[ii, jj] = cx_adj
            CenterY[ii, jj] = cy_adj

            # Ré-extraction de la micro-image ajustée
            LFM[ii, jj] = img[cy_adj-(step//2):cy_adj+(step//2 + 1), cx_adj-(step//2):cx_adj+(step//2 + 1)]

            end = time.perf_counter()
    print(f'Calibration took: {end - start}')

    return LFM, CenterX, CenterY, XX, YY, CenterX_ini, CenterY_ini

def get_max_distance_center2(flooded):
    labeled = label(flooded)
    props = regionprops(labeled)
    best_center = None
    best_score = -1
    for region in props:
        minr, minc, maxr, maxc = region.bbox
        if minr == 0 or minc == 0 or maxr == flooded.shape[0] or maxc == flooded.shape[1]:
            continue  # ignore si ca touche le bord de la sous image
        mask = (labeled == region.label)
        dist = distance_transform_edt(mask)
        max_idx = np.unravel_index(np.argmax(dist), dist.shape)
        score = dist[max_idx]
        if score > best_score:
            best_score = score
            best_center = max_idx
    return best_center if best_center is not None else (ndimage.center_of_mass(flooded))

def compute_LFM(img, CenterX, CenterY, XX, YY):
    start = time.perf_counter()
    step = 15
    LFM = np.zeros([Nx, Ny, 15, 15])
    for ii in range(0, Nx):
        for jj in range(0, Ny):    
            LFM[ii, jj, :, :] = img[int(CenterY[ii, jj])-(step//2):int(CenterY[ii, jj])+(step//2 + 1), 
                                     int(CenterX[ii, jj])-(step//2):int(CenterX[ii, jj])+(step//2 + 1)]
    end = time.perf_counter()
    print(f'LFM Computation took: {end - start}')
    return LFM

def rendered_focus(C,rendered_height, rendered_width, side, radiance, returnVal=False):
    rendered = np.zeros((rendered_height, rendered_width))
    center_uv = int(side/2)
    for u in range(0, side):
        for v in range(0, side):
            shift_x, shift_y = C*(center_uv-u), C*(center_uv-v)
            rendered[:, :] += shift(radiance[:, :, u, v], (shift_x, shift_y))
    final = rendered / (side*side)
    if returnVal:
        return final

In [ ]:
save_dir = "D:/surgele/juin/tests_0406"
file_path0=os.path.join(save_dir,'snap_ref.tiff')       
file_path=os.path.join(save_dir,'snap_0.tiff')
file_path_patternS = sorted(glob.glob(os.path.join(save_dir,'speckles/*.tiff')))
bg_path=os.path.join(save_dir,'snap_bg.tiff')

All LFM persp + zstack

In [ ]:
save_dir = "D:/surgele/juin/tests_0406"
file_path0=os.path.join(save_dir,'snap_ref.tiff')       
file_path=os.path.join(save_dir,'snap_plain.tiff')
file_path_patternS = sorted(glob.glob(os.path.join(save_dir,'speckles/*.tiff')))
bg_path=os.path.join(save_dir,'snap_bg.tiff')

angle = -0.0001 # -0.057 pour les tests de mai
pitch = 14.02 
calibration_image=pre_processing(angle, pitch, file_path0, bg_path=None)
bright=pre_processing(angle,pitch, file_path, bg_path=None)
x0 = 16 # 1er centre à choisir en fonction de calibration_image
y0 = 20
LFM,centerX,centerY,CX,CY, centerX_ini, centerY_ini=calibrate_LFM3(calibration_image, x0, y0)
LFM01=compute_LFM((bright/(calibration_image+1))*np.mean(calibration_image),centerX,centerY, CX, CY)
rendered_height,rendered_width=LFM01.shape[0],LFM01.shape[1]
side=LFM01.shape[2]                                          
radiance=LFM01  

stacked_perspectives = np.stack([radiance[:, :, i, j] for i in range(radiance.shape[3]) for j in range(radiance.shape[3])])

start = time.perf_counter()
rendered_images=[]  # Array to store the volumetric image 
C_values=np.linspace(-2.5,2.5,40)   # Coefficient C that depends on the depth
for C in (C_values):    # Apply Shift and Sum algorithm: set the depth limits and the number of slices (sampling)
    rendered_image=rendered_focus(C, rendered_height, rendered_width, side, radiance, returnVal=True)
    rendered_images.append(rendered_image)
rendered_images_stack=np.stack(rendered_images)
end = time.perf_counter()
print(f'Shift and Sum took: {end - start} s')

40 slices (comme fourier):
Shift and Sum took: 9.43254810000144 s
overall cell: 15.8 s
Shift and Sum took: 9.489312899997458 s
overall cell: 15.8 s
Shift and Sum took: 9.422013300001709 s
overall cell: 15.9 s



80 slices:
Pre-processing took: 0.3854462999988755
Pre-processing took: 0.38949909999792
Calibration took: 5.600926799997978
LFM Computation took: 0.02884999999878346
Shift and Sum took: 18.768576999998913 s
OVERALL CELL : 25.2 s

Pre-processing took: 0.3784602999985509
Pre-processing took: 0.383884400001989
Calibration took: 5.587174699998286
LFM Computation took: 0.029091200001857942
Shift and Sum took: 18.742547099998774 s
OVERALL CELL : 25.1 s

Pre-processing took: 0.3872702000007848
Pre-processing took: 0.3833943999998155
Calibration took: 5.477498000000196
LFM Computation took: 0.027477499999804422
Shift and Sum took: 18.820135499998287 s
OVERALL CELL : 25.1 s

All LFM persp + fourier slice

In [ ]:
save_dir = "D:/surgele/juin/tests_0406"
file_path0=os.path.join(save_dir,'snap_ref.tiff')       
file_path=os.path.join(save_dir,'snap_plain.tiff')
file_path_patternS = sorted(glob.glob(os.path.join(save_dir,'speckles/*.tiff')))
bg_path=os.path.join(save_dir,'snap_bg.tiff')

angle = -0.0001 # -0.057 pour les tests de mai
pitch = 14.02 
calibration_image=pre_processing(angle, pitch, file_path0, bg_path=None)
bright=pre_processing(angle,pitch, file_path, bg_path=None)
x0 = 16 # 1er centre à choisir en fonction de calibration_image
y0 = 20
LFM,centerX,centerY,CX,CY, centerX_ini, centerY_ini=calibrate_LFM3(calibration_image, x0, y0)
LFM01=compute_LFM((bright/(calibration_image+1))*np.mean(calibration_image),centerX,centerY, CX, CY)
rendered_height,rendered_width=LFM01.shape[0],LFM01.shape[1]
side=LFM01.shape[2]                                          
radiance=LFM01  

start = time.perf_counter()
stacked_perspectives = np.stack([radiance[:, :, i, j] for i in range(radiance.shape[3]) for j in range(radiance.shape[3])])
end = time.perf_counter()
print(f'Perspectives generation took: {end - start}')

start = time.perf_counter()
refocus_stack, fft_stack, Z = fourierslice_refocusing.refocus_test(radiance, min=-150, max=150, Padding=True)
end = time.perf_counter()
print(f'Fourier Slicing Refocusing took: {end - start}')

Pre-processing took: 0.3811929000003147
Pre-processing took: 0.38172320000012405
Calibration took: 5.578916100002971
LFM Computation took: 0.027556799999729265
Perspectives generation took: 0.013518899999326095
Fourier Slicing Refocusing took: 1.0312913999987359
OVERALL CELL: 7.4 s

Pre-processing took: 0.38487820000227657
Pre-processing took: 0.3837354999996023
Calibration took: 5.573355299999093
LFM Computation took: 0.027478799998789327
Perspectives generation took: 0.01305069999943953
Fourier Slicing Refocusing took: 0.9866965999972308
OVERALL CELL: 7.4 s

Pre-processing took: 0.39463980000073207
Pre-processing took: 0.3801951999994344
Calibration took: 5.538687399999617
LFM Computation took: 0.02753160000065691
Perspectives generation took: 0.013295500000822358
Fourier Slicing Refocusing took: 0.9647670999984257
OVERALL CELL: 7.3 s

All HiLo-LFM persp + shift and sum

In [ ]:
save_dir = "D:/surgele/juin/tests_0406"
file_path0=os.path.join(save_dir,'snap_ref.tiff')       
file_path=os.path.join(save_dir,'snap_plain.tiff')
file_path_patternS = sorted(glob.glob(os.path.join(save_dir,'speckles/*.tiff'))) # 3 speckles
bg_path=os.path.join(save_dir,'snap_bg.tiff')

angle = -0.0001 # -0.057 pour les tests de mai
pitch = 14.02 
calibration_image=pre_processing(angle, pitch, file_path0, bg_path=None)
bright=pre_processing(angle,pitch, file_path, bg_path=None)
x0 = 16 # 1er centre à choisir en fonction de calibration_image
y0 = 20
LFM,centerX,centerY,CX,CY, centerX_ini, centerY_ini=calibrate_LFM3(calibration_image, x0, y0)
LFM01=compute_LFM((bright/(calibration_image+1))*np.mean(calibration_image),centerX,centerY, CX, CY)
rendered_height,rendered_width=LFM01.shape[0],LFM01.shape[1]
side=LFM01.shape[2]                                          
radiance=LFM01  

print('Now Hilo:')
radiance_patternS = []
side_patternS = []
rendered_height_patternS = []
rendered_width_patternS = []
i = 1
for file_path_pattern in file_path_patternS:
    pattern = pre_processing(angle, pitch, file_path_pattern, bg_path)
    LFM01_pattern=compute_LFM((pattern/(calibration_image+1))*np.mean(calibration_image),centerX,centerY,CX,CY)
    # LFM01_pattern=compute_LFM(cv2.normalize(pattern, None, 0, 65536, cv2.NORM_MINMAX),centerX,centerY,CX,CY)
    rendered_height_pattern,rendered_width_pattern=LFM01_pattern.shape[0],LFM01_pattern.shape[1]
    side_pattern=LFM01_pattern.shape[2]                                          
    radiance_pattern=LFM01_pattern
    persp = radiance_pattern[:, :, 7, 7]
    i += 1
    radiance_patternS.append(radiance_pattern)
    side_patternS.append(side_pattern)
    rendered_height_patternS.append(rendered_height_pattern)
    rendered_width_patternS.append(rendered_width_pattern)

start = time.perf_counter()
hilo_perspectives = []
for i in range(radiance.shape[3]):
    for j in range(radiance.shape[3]):
        radiance_structured = [r[:, :, i, j] for r in radiance_patternS]
        hilo_perspective, _, _, _ = new_hilo1.basic_hilo(radiance[:, :, i, j], radiance_structured)
        hilo_perspectives.append(hilo_perspective)
hilo_perspectives_stack = np.stack(hilo_perspectives)
end = time.perf_counter()
print(f'HiLo Perspectives generation took: {end - start}')

U = V = int(np.sqrt(hilo_perspectives_stack.shape[0]))
X, Y = hilo_perspectives_stack.shape[1], hilo_perspectives_stack.shape[2]
radiance_hilo = hilo_perspectives_stack.reshape(U, V, X, Y).transpose(2, 3, 0, 1)

rendered_images_hilo=[]  # Array to store the volumetric image 
C_values=np.linspace(-1.5,1.5,40)   # Coefficient C that depends on the depth
for C in (C_values):    # Apply Shift and Sum algorithm: set the depth limits and the number of slices
    rendered_image_hilo=rendered_focus(C, rendered_height, rendered_width, side, radiance_hilo, returnVal=True)
    rendered_images_hilo.append(rendered_image_hilo)
rendered_images_stack_hilo=np.stack(rendered_images_hilo)

Pre-processing took: 0.38023940000130096
Pre-processing took: 0.3812873999995645
Calibration took: 5.569188900000881
LFM Computation took: 0.02754339999955846
Now Hilo:
Pre-processing took: 0.38818560000072466
LFM Computation took: 0.027472699999634642
Pre-processing took: 0.3912065000004077
LFM Computation took: 0.027311599998938618
Pre-processing took: 0.3859755000012228
LFM Computation took: 0.026935199999570614
HiLo reconstruction took: 0.014775699997699121
HiLo reconstruction took: 0.014583000000129687
HiLo reconstruction took: 0.012090099997294601
HiLo reconstruction took: 0.012897900000098161
...
HiLo reconstruction took: 0.011652999997750157
HiLo reconstruction took: 0.01580400000239024
HiLo Perspectives (225) generation took: 2.8994628000000375
OVERALL CELL: 28.1 s

HiLo Perspectives generation took: 2.9092525000014575
OVERALL CELL: 28.1 s

HiLo Perspectives generation took: 2.8939372999993793
OVERALL CELL: 28.1 s

All HiLo-LFM persp + Fourier Slice

In [ ]:
save_dir = "D:/surgele/juin/tests_0406"
file_path0=os.path.join(save_dir,'snap_ref.tiff')       
file_path=os.path.join(save_dir,'snap_plain.tiff')
file_path_patternS = sorted(glob.glob(os.path.join(save_dir,'speckles/*.tiff')))
bg_path=os.path.join(save_dir,'snap_bg.tiff')

angle = -0.0001 # -0.057 pour les tests de mai
pitch = 14.02 
calibration_image=pre_processing(angle, pitch, file_path0, bg_path=None)
bright=pre_processing(angle,pitch, file_path, bg_path=None)
x0 = 16 # 1er centre à choisir en fonction de calibration_image
y0 = 20
LFM,centerX,centerY,CX,CY, centerX_ini, centerY_ini=calibrate_LFM3(calibration_image, x0, y0)
LFM01=compute_LFM((bright/(calibration_image+1))*np.mean(calibration_image),centerX,centerY, CX, CY)
rendered_height,rendered_width=LFM01.shape[0],LFM01.shape[1]
side=LFM01.shape[2]                                          
radiance=LFM01  

radiance_patternS = []
side_patternS = []
rendered_height_patternS = []
rendered_width_patternS = []
i = 1
for file_path_pattern in file_path_patternS:
    pattern = pre_processing(angle, pitch, file_path_pattern, bg_path)
    LFM01_pattern=compute_LFM((pattern/(calibration_image+1))*np.mean(calibration_image),centerX,centerY,CX,CY)
    # LFM01_pattern=compute_LFM(cv2.normalize(pattern, None, 0, 65536, cv2.NORM_MINMAX),centerX,centerY,CX,CY)
    rendered_height_pattern,rendered_width_pattern=LFM01_pattern.shape[0],LFM01_pattern.shape[1]
    side_pattern=LFM01_pattern.shape[2]                                          
    radiance_pattern=LFM01_pattern
    persp = radiance_pattern[:, :, 7, 7]
    # tiff.imwrite(os.path.join(save_dir, f'perspective_centrale_speckle{i}.tiff'), persp)
    print("Array shape:", radiance_pattern.shape)
    print("Max value:", np.max(radiance_pattern))
    i += 1
    radiance_patternS.append(radiance_pattern)
    side_patternS.append(side_pattern)
    rendered_height_patternS.append(rendered_height_pattern)
    rendered_width_patternS.append(rendered_width_pattern)

hilo_perspectives = []
for i in range(radiance.shape[3]):
    for j in range(radiance.shape[3]):
        radiance_structured = [r[:, :, i, j] for r in radiance_patternS]
        hilo_perspective, _, _, _ = new_hilo1.basic_hilo(radiance[:, :, i, j], radiance_structured)
        hilo_perspectives.append(hilo_perspective)
hilo_perspectives_stack = np.stack(hilo_perspectives)

U = V = int(np.sqrt(hilo_perspectives_stack.shape[0]))
X, Y = hilo_perspectives_stack.shape[1], hilo_perspectives_stack.shape[2]
radiance_hilo = hilo_perspectives_stack.reshape(U, V, X, Y).transpose(2, 3, 0, 1)

hilo_refocus_stack, hilo_fft_stack, Z = fourierslice_refocusing.refocus_test(radiance_hilo, min=150, max=150, Padding=False)

OVERALL CELL: 10.7 s

OVERALL CELL: 10.7 s

OVERALL CELL: 10.6 s